## Retrieving Relevant Context using Word Embeddings

### Work Breakdown

- [x] Find pre-trained embeddings in Bulgarian
- [x] Load wikipedia documents
- [ ] Chunk documents into sentences
- [ ] Load embedding vectors
    - [ ] Build an index from word to vector for mapping words to vectors
    - [ ] Build an index from vector to vector for nearest neighbour search
- [ ] Map each sentence/passage to an embedding vector
    - [ ] Paremeterize the aggregation function - avg, max, etc.
- [ ] Load questions and answer options from `bg_rc-v1.0.json`
- [ ] For each question find the nearest contexts (passages, sentences)
- [ ] Paremeterize each step and expose configuration for grid search

In [61]:
import json
import numpy as np
import pandas as pd
import ujson
import glob
import ngtpy

from polyglot.text import Text, Word

In [2]:
embeddings_index = {}
f = open('../data/conceptnet-numberbatch/bg.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 20870 word vectors.


In [3]:
question = 'Всички организми, населяващи една и съща територия и свързани помежду си с разнообразни отношения, образуват'

# TODO: tokenize question

question_tokens = ['всички', 'организми', 'населяващи', 'една', 'и', 'съща', 'територия', 'и', 'свързани', 'помежду', 'си', 'с', 'разнообразни', 'отношения', 'образуват']

In [4]:
list(map(lambda x: embeddings_index[x] if x in embeddings_index else [], question_tokens))

[array([ 0.098 ,  0.079 , -0.0825,  0.0258, -0.0274, -0.041 ,  0.111 ,
        -0.0259, -0.0237,  0.1066,  0.055 ,  0.0541,  0.1162, -0.0235,
         0.0565, -0.0536,  0.1176,  0.0051,  0.0862,  0.0844, -0.0372,
        -0.0334,  0.1121,  0.0145,  0.012 , -0.006 ,  0.0206,  0.0198,
        -0.0097,  0.0462,  0.0439, -0.0868, -0.0408, -0.0386, -0.0623,
         0.0876, -0.0016, -0.0391,  0.1291,  0.0486,  0.0431,  0.1008,
        -0.0473, -0.0093, -0.0071, -0.1366,  0.0423,  0.0227,  0.0444,
        -0.0779,  0.0241,  0.0612,  0.0402,  0.0054, -0.0035, -0.0239,
        -0.0376, -0.1475,  0.0587,  0.087 ,  0.0421,  0.0342, -0.099 ,
         0.0949,  0.0104,  0.0838, -0.0618,  0.0014, -0.0029, -0.0119,
         0.0162,  0.0591,  0.0014,  0.0569,  0.0372,  0.0888, -0.0429,
         0.1308,  0.0406, -0.0649, -0.0388,  0.1122, -0.0268,  0.0776,
        -0.0354, -0.0852,  0.0552,  0.0823, -0.1194,  0.077 ,  0.0422,
         0.0739,  0.0633,  0.035 ,  0.1083,  0.0161, -0.0607, -0.0389,
      

In [5]:
documents_glob = '../data/context/bgwiki.json/*/wiki_*'

In [6]:
df = pd.DataFrame()
for path in glob.glob(documents_glob):
    records = map(ujson.loads, open(path))
    df = df.append(pd.DataFrame.from_records(records))

In [7]:
df = df.set_index('id')
df = df.drop('url', axis=1)
df

,text,title
id,,
549994,Програма за политическите отношения на сърбо-б...,Програма за политическите отношения на сърбо-б...
549996,Кастел дел Монте (село)\n\nКастѐл дел Мо̀нте (...,Кастел дел Монте (село)
550003,У дома\n\nУ дома () е американски 3D компютърн...,У дома
550011,Нефтохимик 2010\n\nБК „Нефтохимик 2010“ е бълг...,Нефтохимик 2010
550012,Кървав залез\n\n„Кървав залез“ () е гръцки дра...,Кървав залез
550013,"Ноктопластика\n\nНоктопластика е удължаването,...",Ноктопластика
550016,Бояна Донева\n\nБояна Георгиева Донева е бълга...,Бояна Донева
550018,Никола Гандев\n\nНикола Нонев Гандев е българс...,Никола Гандев
550021,Бернар Лепти\n\nБернар Лепти () е френски исто...,Бернар Лепти


In [13]:
text = df[df.index == '373139']['text'].iloc[0]
text

'Горене\n\nГорене или изгаряне е окислително-редукционен процес, при който се излъчва енергия под формата на топлина и светлина. Горенето също е екзотермична реакция.\n\nКато цяло под понятието горене се разбира окисление на даден материал с кислород под формата на огън. В Химията съществуват реакции под името горене, които се осъществяват без кислород. Такава е реакцията за получаване на флуороводород от флуор и водород. При тази реакция флуорът замества кислорода като окислител.\n\nОтделяне на топлина може да доведе до производството на светлина под формата на тлеене или пламък. Горивата често включват органични съединения (особено въглеводород) в газовата, течната или твърдата фаза.\n\nГоренето е първият химически процес, овладян от човека. Усвояването на огъня изиграва ключова роля в развитието на човешката цивилизация. Огънят открива пред хората възможността за топлинна обработка на храна и отопление на жилищата, а впоследствие\xa0– развитието на металургията, енергетиката и създа

In [16]:
Text(text).sentences

[Sentence("Горене"),
 Sentence("Горене или изгаряне е окислително-редукционен процес, при който се излъчва енергия под формата на топлина и светлина."),
 Sentence("Горенето също е екзотермична реакция."),
 Sentence("Като цяло под понятието горене се разбира окисление на даден материал с кислород под формата на огън."),
 Sentence("В Химията съществуват реакции под името горене, които се осъществяват без кислород."),
 Sentence("Такава е реакцията за получаване на флуороводород от флуор и водород."),
 Sentence("При тази реакция флуорът замества кислорода като окислител."),
 Sentence("Отделяне на топлина може да доведе до производството на светлина под формата на тлеене или пламък."),
 Sentence("Горивата често включват органични съединения (особено въглеводород) в газовата, течната или твърдата фаза."),
 Sentence("Горенето е първият химически процес, овладян от човека."),
 Sentence("Усвояването на огъня изиграва ключова роля в развитието на човешката цивилизация."),
 Sentence("Огънят откри

In [57]:
Text(text).sentences[1].dict

{'raw': 'Горене или изгаряне е окислително-редукционен процес, при който се излъчва енергия под формата на топлина и светлина.',
 'start': 8,
 'end': 126,
 'entities': [],
 'tokens': WordList(['Горене', 'или', 'изгаряне', 'е', 'окислително', '-', 'редукционен', 'процес', ',', 'при', 'който', 'се', 'излъчва', 'енергия', 'под', 'формата', 'на', 'топлина', 'и', 'светлина', '.']),
 'words': WordList(['Горене', 'или', 'изгаряне', 'е', 'окислително', '-', 'редукционен', 'процес', ',', 'при', 'който', 'се', 'излъчва', 'енергия', 'под', 'формата', 'на', 'топлина', 'и', 'светлина', '.']),
 'pos_tags': [('Горене', 'NOUN'),
  ('или', 'CONJ'),
  ('изгаряне', 'NOUN'),
  ('е', 'AUX'),
  ('окислително', 'ADJ'),
  ('-', 'PUNCT'),
  ('редукционен', 'ADJ'),
  ('процес', 'NOUN'),
  (',', 'PUNCT'),
  ('при', 'ADP'),
  ('който', 'PRON'),
  ('се', 'PRON'),
  ('излъчва', 'VERB'),
  ('енергия', 'NOUN'),
  ('под', 'ADP'),
  ('формата', 'NOUN'),
  ('на', 'ADP'),
  ('топлина', 'NOUN'),
  ('и', 'CONJ'),
  ('светл

In [69]:
dim = 300
objects = list(embeddings_index.values())

ngtpy.create("numberbatch", dim)
index = ngtpy.Index("numberbatch")
index.batch_insert(objects)
index.save()

In [70]:
query = objects[0]
result = index.search(query, 3)

for i, o in enumerate(result) :
    print(str(i) + ": " + str(o[0]) + ", " + str(o[1]))
    object = index.get_object(o[0])
    print(object)

0: 0, 0.0
[-0.13050000369548798, -0.3409999907016754, -0.10729999840259552, -0.13279999792575836, 0.06279999762773514, -0.05770000070333481, -0.039900001138448715, -0.006099999882280827, 0.03669999912381172, -0.016499999910593033, -0.0681999996304512, -0.10599999874830246, 0.07240000367164612, 0.20839999616146088, 0.04699999839067459, 0.11469999700784683, -0.03460000082850456, -0.0925000011920929, -0.003100000089034438, -0.03840000182390213, 0.005900000222027302, 0.03909999877214432, -0.0746999979019165, -0.029400000348687172, -0.05480000004172325, -0.04280000180006027, -0.13779999315738678, 0.0714000016450882, -0.07050000131130219, 0.0560000017285347, 0.0551999993622303, -0.08290000259876251, 0.06930000334978104, -0.08940000087022781, -0.002300000051036477, -0.07289999723434448, 0.08190000057220459, 0.09939999878406525, -0.01360000018030405, 0.03689999878406525, 0.06340000033378601, 0.08749999850988388, 0.009999999776482582, -0.038600001484155655, -0.05889999866485596, -0.036200001835

In [71]:
len(objects)

20870